In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 4.7 MB/s 
     |████████████████████████████████| 209 kB 42.7 MB/s 
     |████████████████████████████████| 81 kB 5.9 MB/s 
     |████████████████████████████████| 78 kB 4.1 MB/s 
     |████████████████████████████████| 146 kB 15.8 MB/s 
     |████████████████████████████████| 112 kB 4.1 MB/s 
     |████████████████████████████████| 49 kB 3.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=7aa2df9660192af201659c7491ca553a6db4f13e601d99c46e0674ead85674da
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
class CFG:
    output_dir = '../output'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
exps = [
'exp039', # anferico/bert-for-patents
'exp094',
'exp122',
'exp127',
'exp080', # facebook/muppet-roberta-large
'exp132',
'exp064', # funnel-transformer/large
'exp126',
'exp073',
'exp069', # microsoft/cocolm-large
'exp123',
'exp068', # microsoft/deberta-v3-base
'exp051', # microsoft/deberta-v3-large
'exp077',
'exp090',
'exp119',
'exp124',
'exp128',
'exp067', # roberta-large
'exp115', # google/electra-large-discriminator
'exp131',
'exp129',
'exp130'
]

scores = {}
oof_dfs = {}

for exp_ in exps:
  scores[exp_] = pd.read_csv(f'{CFG.output_dir}/{exp_}/scores.csv')
  oof_dfs[exp_] = pd.read_csv(f'{CFG.output_dir}/{exp_}/oof_df.csv')

In [ ]:
def run_all_return_study(exps_tmp):
  df = oof_dfs[list(oof_dfs.keys())[0]][['fold', 'score']].copy()
  for exp_ in exps_tmp:
    df[f'{exp_}'] = oof_dfs[exp_]['preds']
  mean_ = np.nanmean(df.drop(['fold', 'score'], axis='columns').values, axis=1)

  def val(trial):
    weights = []
    for i in exps_tmp:
      weights.append(trial.suggest_float(f'{i}', 0, 1))
    sum_ = np.nansum(df[list(exps_tmp)].values*weights, axis=1) / len(exps_tmp)
    score = np.corrcoef(df['score'], sum_)[0][1]
    return score

  optuna.logging.disable_default_handler()
  study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
  study.optimize(val, n_trials=256)
  return study

In [ ]:
drop_list = ['exp126', 'exp068', 'exp039']
exp_all = [i for i in exps if i not in drop_list]
study = run_all_return_study(exp_all)

In [ ]:
print(study.best_trial.number)
print(study.best_trial.values)
print(study.best_trial.params)

255
[0.8534998122621883]
{'exp094': 0.36170636732895645, 'exp122': 0.858818966307967, 'exp127': 0.9496997677405316, 'exp080': 0.07750493560602226, 'exp132': 0.4389368360537185, 'exp064': 6.026089818115393e-05, 'exp073': 0.26697121145720687, 'exp069': 0.4548710334368818, 'exp123': 0.32000672265278796, 'exp051': 0.41013542927880803, 'exp077': 0.3859025419537993, 'exp090': 0.9631566085835841, 'exp119': 0.6013280290660092, 'exp124': 0.6296836244301492, 'exp128': 0.4329802078271134, 'exp067': 0.028160755539509415, 'exp115': 0.3392619616877845, 'exp131': 0.2503211193265256, 'exp129': 0.35524533425827315, 'exp130': 0.5523234189681866}
